In [7]:
import random

#### We first need classes for representing edges and nodes.

In [37]:
class Node: # a class for representing a node as a single unique integer
    
    def __init__(self, id):
        self.id = id
        
    def __str__(self):
        return str(self.id)
        
    def __eq__(self, other):
        if type(other) == int:
            other = Node(other)
        return self.id == other.id
        
class Edge: # a class for representing an edge as a pair of nodes in a graph
    
    def __init__(self, n1, n2, weight=1):          # initializing the nodes in the edge class
        self.n1 = n1
        self.n2 = n2
        self.weight = weight                       # the default argument for weight is 1 (call it the existence weight)
        
    def __str__(self):                             # turning the Edge object into a string
        return '(' + str(self.n1) + ', ' + str(self.n2) + ')'
        
    def __eq__(self, other):                       # easy to implement equality method between two edges
        return self.n1 == other.n1 and self.n2 == other.n2

#### Below, we have classes for each of three different graph representations: as a list of edges, as a matrix of adjacency pairings, and as a list of adjacencies (each index maps to a set of adjacent nodes.

In [38]:
class EdgeList(object): # a simple class for representing a graph as a list of edges
    
    def __init__(self):                # initialization of the EdgeList class
        self.E = []                    # instantiate empty list to hold the edges
     
    def addEdge(self, edge):           # add an edge to the edge list
        self.E.append(edge)
        
    def isEdge(self, edge):            # does this edge appear in the graph? this naive algorithm takes O(|E|) time
        for e in self.E:               # cycle through all of the edges in the edge list
            if e == edge:              # check for equality
                return True
        return False                   # if none of the edges match, it is not contained in the graph
    
    def getNeighbors(self, node):      # get all nodes adjacent to this node. this algorithm runs in O(|E|) time
        neighbors = set()
        for edge in self.E:                 # loop through all edges in edge list
            if edge.n1 == node:             # if the edge contains the node as an endpoint
                neighbors.update([edge.n2.id])     # add it to the set of neighbors
            elif edge.n2 == node:
                neighbors.update([edge.n1.id])
        return neighbors
            
    def __str__(self):                 # turning an EdgeList object into a string
        rep = '['
        for edge in self.E[:-1]:
            rep += edge.__str__() + ', '
        return rep + self.E[-1].__str__() + ']'

In [164]:
class AdjacencyMatrix(object):                              # a simple class for representing a graph as a matrix of connection weights
    
    def __init__(self, V):                          # initialization of the AdjacencyMatrix class
        self.matrix = [[0 for d in range(V)] for d in range(V)]
        self.V = V                                  # keep track of the dimension of the matrix (number of nodes)
        
    def addEdge(self, edge):                        # add an edge to the adjacency matrix
        self.matrix[edge.n1.id][edge.n2.id] = edge.weight
        
    def isEdge(self, edge):                         # does this edge appear in the graph? this algorithm takes 0(1) time
        return self.matrix[edge.n1.id][edge.n2.id]  # test for non-zero
    
    def getNeighbors(self, node):                   # get all nodes adjacent to this node. this algorithm runs in O(|V|) time
        neighbors = set()
        for i in range(self.V):                          # for every node in the graph
            if self.matrix[node][i]:                # test for non-zero neighboring entries
                neighbors.update([i])                 # add the node to the graph if it is non-zero
        return neighbors
    
    def __str__(self):                              # turning an AdjacencyMatrix object into a string
        rep = ''
        for row in self.matrix:
            for entry in row:
                rep += str(entry) + ' '
            rep += '\n'
        return rep

In [256]:
class AdjacencyList(object):                                # a simple class for representing a graph as a list of adjancencies
    
    def __init__(self, V):                          # initialization of the AdjacencyList class
        self.V = V
        self.adj_list = dict((i, set()) for i in range(V))        
        
    def addEdge(self, edge):                        # add an edge to the adjacency list
        self.adj_list[edge.n1.id].add(edge.n2.id)   
        
    def isEdge(self, edge):                         # does this edge appear in the graph? worst case, O(n) time, average case, O(1/|V|)
        return edge.n2.id in self.adj_list[edge.n1.id]
    
    def getNeighbors(self, node):                   # get all nodes adjacent to this node. this algorithm runs in O(1) time
        return self.adj_list[node]                  # simply return the set indexed by the passed in node index!
    
    def __str__(self):                              # turning an AdjacencyList object into a string
        rep = ''
        for i in range(self.V):
            rep += str(i) + ': '
            for j in self.adj_list[i][:-1]:
                rep += j + ', '
            rep += self.adj_list[i][-1]
            rep += '\n'

#### Now, we need classes for both undirected and directed graph implementations.

In [257]:
class Graph(object): # superclass for creating either a directed or undirected graph
    
    def __init__(self, representation, num_V, num_E, V=set(), E=set()):
        self.V = V
        self.E = E
        
        if representation == 'edge_list':
            self.G = EdgeList()
        elif representation == 'adjacency_matrix':
            self.G = AdjacencyMatrix(num_V)
        elif representation == 'adjacency_list':
            self.G = AdjacencyList(num_V)
            
    def getNeighbors(self, node):                       # method to get neighbors of a node
        neighbors = self.G.getNeighbors(node)
        if neighbors != None:                                   # call underlying graph representation's implementation
            return neighbors
        return set()
        
    def DFS(self, start):                               # iterative depth-first search algorithm for undirected graphs
        visited = set()                                 # we keep track of which nodes we've visited thus far. no duplicates allowed
        stack = [start]                                 # we keep as a stack (implemented as a list) for the LIFO behavior this algorithm requires
        while stack:                                    # while this list (stack) is non-empty
            node = stack.pop()                          # we pop the topmost element of the list (LIFO)
            if node not in visited:                     # if we haven't seen this node before
                visited.add(node)                       # add it to the visited list
                neighbors = self.getNeighbors(node)
                if neighbors:
                    stack.extend(self.getNeighbors(node).difference(visited))    # add all unvisited neighboring nodes to the list (stack) 
        return sorted(visited)                          # this returns the set of visited nodes after we have emptied the stack
    
    def recursiveDFS(self, start, visited=None):        # recursive depth-first search algorithm for undirected graphs
        if visited == None:                             # circumventing a common Python error dealing with default values being created only once
            visited = set()
        visited.add(start)                              # add the starting node to the set of visited nodes
        neighbors = self.getNeighbors(start)
        if neighbors:
            for neighbor in neighbors.difference(visited):  # for each neighbor of the starting node
                self.recursiveDFS(neighbor, visited)        # treat each neighbor as a new starting node, and call the DFS routine again
        return sorted(visited)                              # this returns all visited nodes after we have completed the last recursive call
            

class UndirectedGraph(Graph): # class for creating undirected graphs of 3 different possible representations

    def __init__(self, representation, num_V, num_E, V=set(), E=set()):
        Graph.__init__(self, representation, num_V, num_E, V=set(), E=set())
        
        if not V:                                           # if no edges, nodes were given in the constructor
            # loop through |E| to randomly assign edges on the graph
            for i in range(num_E):
                n1, n2 = Node(random.randrange(0, num_V)), Node(random.randrange(0, num_V))    # randomly sample two vertices from [0, V-1]
                if n1 == n2:                                # if they're equal, decrement the loop and try again
                    i = i - 1                               # why? no self-directed edges (trivial)
                    continue

                edge1, edge2 = Edge(n1, n2), Edge(n2, n1)   # undirected graphs have only symmetric edges
                if self.G.isEdge(edge1):                    # if the edge is already in the graph
                    i = i - 1                               # we decrement the loop counter and try again
                    continue
                self.G.addEdge(edge1)                       # so we add both (n1, n2) and (n2, n1) to the representation
                self.G.addEdge(edge2)
        else:                                               # edges, nodes were given in the constructor
            for edge in E:                                  # we loop through each edge 
                self.G.addEdge(edge)                        # and add the provided edge to the graph data structure
            
    def DFS(self, start):                                   # iterative depth-first search algorithm for undirected graphs
        return super(UndirectedGraph, self).DFS(start)      # call superclass' method
    
    def recursiveDFS(self, start, visited=None):            # recursive depth-first search algorithm for undirected graphs
        return super(UndirectedGraph, self).recursiveDFS(start, visited)  # call superclass' method
    
    def BFS(self, start):
        pass
    
    def __str__(self):                                  # turn UndirectedGraph object into a string
        return self.G.__str__()                         # call underlying representation's __str__ method

In [258]:
class DirectedGraph(Graph):
    # a class for creating directed graphs of 3 different possible representations

    def __init__(self, graph_type, num_V, num_E, V=set(), E=set()):
        Graph.__init__(self, graph_type, num_V, num_E, V=set(), E=set())
        
        if not V:                                           # if no edges, nodes were given in the constructor
            # loop through |E| to randomly assign edges on the graph
            for i in range(num_E):
                n1, n2 = Node(random.randrange(0, num_V)), Node(random.randrange(0, num_V))    # randomly sample two vertices from [0, V-1]
                if n1 == n2:                                # if they're equal, decrement the loop and try again
                    i = i - 1                               # why? no self-directed edges (trivial)
                    continue

                edge1 = Edge(n1, n2)                        # directed graphs do not necessarily have symmetric edges
                if self.G.isEdge(edge1):                    # if the edge is already in the graph
                    i = i - 1                               # we decrement the loop counter and try again
                    continue
                self.G.addEdge(edge1)                       # so we add both (n1, n2) and (n2, n1) to the representation
        else:                                               # edges, nodes were given in the constructor
            for edge in E:                                  # we loop through each edge 
                self.G.addEdge(edge)                        # and add the provided edge to the graph data structure
        
    def DFS(self, start):
        return super(DirectedGraph, self).DFS(start)        # call superclass' method
    
    def BFS(self, start):
        return super(DirectedGraph, self).recursiveDFS(start, visited)  # call superclass' method
    
    def __str__(self):                                      # turn DirectedGraph object into a string
        return self.G.__str__()                             # call underlying representation's __str__ method

In [289]:
G = DirectedGraph('adjacency_list', 100, 250)

In [300]:
connectivities = []
num_trials = 50

for i in [10, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900]:
    sum = 0
    random_node = random.choice(range(100))
    for j in range(num_trials):
        G = DirectedGraph('adjacency_list', 100, i)
        sum += len(G.recursiveDFS(random_node))
    connectivities.append(sum / num_trials)
    
print connectivities

[1, 1, 6, 25, 64, 78, 89, 92, 96, 98, 99, 99, 99, 99, 99, 99, 99, 99, 100]
